In [ ]:
### load dataset and packages
from sklearn.decomposition import NMF
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import category_encoders
from sklearn.cluster import KMeans
df = pd.read_pickle("cleaned_data.pkl")


In [ ]:
### create dummy data frame and binarize continuous features to qunitiles
df_dummy = df.copy()
conF = ["Age","first_contract_date","gross_income","seniority_time"]

for m in conF:
    df_dummy[m] = pd.qcut(df_dummy[m],5,labels=[0,1,2,3,4]).to_numpy()
    



In [ ]:
### reduce data set to features in question 
coll = df_dummy.loc[:,:"Direct_Debit"].columns
coll = coll.to_list()+["leave_ones","new_ones"]

df_dummy = df_dummy.loc[coll]

### features of persons
df_f = df_dummy.loc[:,:"customer_segment"]
product_basket = df_dummy.loc[:,"Saving_Account":"Direct_Debit"]



In [ ]:
### reduce df_f to unique entries

df_f.reset_index(level=1,inplace=True)
df_f.drop_duplicates(subset="id",inplace=True)

In [ ]:
### apply binary encoder to obtain sparse matrix consisting of 0 and 1
en = BinaryEncoder()
en.fit(df_f)
df_f_trans = en.transform(df_f)

In [ ]:
### apply NMF to sparse matrix
nmf = NMF(n_components=4)
feat_components = nmf.fit_transform(df_f_trans)



In [ ]:
### apply KMeans to components to obtain unambigous cluster assignments 
km = KMeans(n_components=4)
feat_components = km.fit_transform(feat_components)

In [ ]:
### now the same for the product basket; here we use the whole dataset without BinaryEncoding
nmf = NMF(n_components=5)
product_components =nmf.fit_transform(product_basket)




In [ ]:
km = Kmeans(n_compnents = 4)
product_components = km.fit_transform(product_components)

In [ ]:
### heatmap of product cluster
product_basket["clusters"] = product_components

colv = product_basket.columns.to_list()
colv.remove("clusters")
product_clusters = product_basket.groupby("clusters")[colv].value_counts(normalize=True)
sns.heatmap(product_clusters,cmap="Blues")

